In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import csv

In [2]:
# 복수 종목 실시간 조회 샘플 (조회는 없고 실시간만 있음)
class CpEvent:
    def set_params(self, client):
        self.client = client
 
    def OnReceived(self):
        code = self.client.GetHeaderValue(0)  # 초
        name = self.client.GetHeaderValue(1)  # 초
        timess = self.client.GetHeaderValue(18)  # 초
        exFlag = self.client.GetHeaderValue(19)  # 예상체결 플래그
        cprice = self.client.GetHeaderValue(13)  # 현재가
        diff = self.client.GetHeaderValue(2)  # 대비
        cVol = self.client.GetHeaderValue(17)  # 순간체결수량
        vol = self.client.GetHeaderValue(9)  # 거래량
 
        if (exFlag == ord('1')):  # 동시호가 시간 (예상체결)
            print("실시간(예상체결)", name, timess, "*", cprice, "대비", diff, "체결량", cVol, "거래량", vol)
        elif (exFlag == ord('2')):  # 장중(체결)
            print("실시간(장중 체결)", name, timess, cprice, "대비", diff, "체결량", cVol, "거래량", vol)

In [3]:
class CpStockCur:
    def Subscribe(self, code):
        self.objStockCur = win32com.client.Dispatch("DsCbo1.StockCur")
        handler = win32com.client.WithEvents(self.objStockCur, CpEvent)
        self.objStockCur.SetInputValue(0, code)
        handler.set_params(self.objStockCur)
        self.objStockCur.Subscribe()
 
    def Unsubscribe(self):
        self.objStockCur.Unsubscribe()

In [4]:
class CpMarketEye:
    def Request(self, codes, rqField):
        # 연결 여부 체크
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            return False
 
        # 관심종목 객체 구하기
        objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        # 요청 필드 세팅 - 종목코드, 종목명, 시간, 대비부호, 대비, 현재가, 거래량
        # rqField = [0,17, 1,2,3,4,10]
        objRq.SetInputValue(0, rqField) # 요청 필드
        objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        objRq.BlockRequest()
 
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = objRq.GetDibStatus()
        rqRet = objRq.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        cnt  = objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            rpCode = objRq.GetDataValue(0, i)  # 코드
            rpName = objRq.GetDataValue(1, i)  # 종목명
            rpTime= objRq.GetDataValue(2, i)  # 시간
            rpDiffFlag = objRq.GetDataValue(3, i)  # 대비부호
            rpDiff = objRq.GetDataValue(4, i)  # 대비
            rpCur = objRq.GetDataValue(5, i)  # 현재가
            rpVol = objRq.GetDataValue(6, i)  # 거래량
            print(rpCode, rpName, rpTime,  rpDiffFlag, rpDiff, rpCur, rpVol)
  
        return True

In [5]:
class MyWindow(QMainWindow):
 
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PLUS API TEST")
        self.setGeometry(300, 300, 300, 150)
        self.isSB = False
        self.objCur = []
 
        btnStart = QPushButton("요청 시작", self)
        btnStart.move(20, 20)
        btnStart.clicked.connect(self.btnStart_clicked)
 
        btnStop = QPushButton("요청 종료", self)
        btnStop.move(20, 70)
        btnStop.clicked.connect(self.btnStop_clicked)
 
        btnExit = QPushButton("종료", self)
        btnExit.move(20, 120)
        btnExit.clicked.connect(self.btnExit_clicked)
 
    def StopSubscribe(self):
        if self.isSB:
            cnt = len(self.objCur)
            for i in range(cnt):
                self.objCur[i].Unsubscribe()
            print(cnt, "종목 실시간 해지되었음")
        self.isSB = False
 
        self.objCur = []
 
    def btnStart_clicked(self):
        self.StopSubscribe();
 
        # 요청 종목 배열
        codes = ["A079970", "A108860", "A300080", "A338220", "A347860", "A095910"]
        # 요청 필드 배열 - 종목코드, 시간, 대비부호 대비, 현재가, 거래량, 종목명
        rqField = [0, 1, 2, 3, 4, 10, 17]  #요청 필드
        objMarkeyeye = CpMarketEye()
        if (objMarkeyeye.Request(codes, rqField) == False):
            exit()
 
        cnt = len(codes)
        for i in range(cnt):
            self.objCur.append(CpStockCur())
            self.objCur[i].Subscribe(codes[i])
 
        print("빼기빼기================-")
        print(cnt , "종목 실시간 현재가 요청 시작")
        self.isSB = True
 
    def btnStop_clicked(self):
        self.StopSubscribe()
 
 
    def btnExit_clicked(self):
        self.StopSubscribe()
        exit()

def save_to_csv(self, filename, data):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['종목코드', '종목명', '시간', '대비부호', '대비', '현재가', '거래량'])
        for row in data:
            writer.writerow(row)

In [6]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()

통신상태 0 0027 조회가 완료되었습니다.(market.eye)
A079970 942 53 -51 1027 465262 투비소프트
A108860 942 50 770 18670 388387 셀바스AI
A300080 942 53 -1000 30350 14942 플리토
A338220 942 50 1300 35550 529745 뷰노
A347860 942 53 -130 9950 44695 알체라
A095910 942 50 10 2665 10043 에스에너지
빼기빼기================-
6 종목 실시간 현재가 요청 시작
실시간(장중 체결) 뷰노 94247 35550 대비 1300 체결량 2 거래량 529747
실시간(장중 체결) 뷰노 94247 35550 대비 1300 체결량 1 거래량 529748
실시간(장중 체결) 플리토 94247 30350 대비 -1000 체결량 32 거래량 14974
실시간(장중 체결) 뷰노 94248 35700 대비 1450 체결량 261 거래량 530009
실시간(장중 체결) 셀바스AI 94249 18670 대비 770 체결량 25 거래량 388412
실시간(장중 체결) 뷰노 94250 35600 대비 1350 체결량 1 거래량 530010
실시간(장중 체결) 뷰노 94250 35550 대비 1300 체결량 1 거래량 530011
실시간(장중 체결) 뷰노 94250 35500 대비 1250 체결량 34 거래량 530045
실시간(장중 체결) 뷰노 94250 35500 대비 1250 체결량 256 거래량 530301
실시간(장중 체결) 뷰노 94250 35500 대비 1250 체결량 18 거래량 530319
실시간(장중 체결) 셀바스AI 94251 18680 대비 780 체결량 53 거래량 388465
실시간(장중 체결) 셀바스AI 94251 18670 대비 770 체결량 100 거래량 388565
실시간(장중 체결) 뷰노 94252 35500 대비 1250 체결량 13 거래량 530332
실시간(장중 체결) 뷰노 94253 35